## Implement Graph

In [6]:
####################
# Implement Graph  #
####################

### Graph Data Structure ###

class Graph:
    """Representation of a simple graph using an adjacency map."""
    
    # ----- nested Vertex class ----- #
    class Vertex:
        """Lightweight vertext structure for a graph."""
        __slots__ = '_element'
        
        def __init__(self, x):
            """Do not call constructor directly. Use Graph's inser_vertex(x)."""
            self._element = x
        
        def element(self):
            """Return element assocaited with this vertex."""
            return self._element
        
        def __hash__(self):     # will allow vertex to be a map/set key
            return hash(id(self))
        
    # ----- nested Edge class ----- #
    class Edge:
        """Lightweight edge strcture for a graph."""
        __slot__ = '_origin', '_destination', '_element'
        
        def __init__(self, u, v, x):
            """Do not call constructor directly. Use Graph's insert_edge(u,v,x)."""
            self._origin = u
            self._destination = v
            self._element = x
            
        def endpoints(self):
            """Return (u,v) tuple for vertices u and v."""
            return (self._origin, self._destination)
        
        def opposite(self, v):
            """Return the vertex that is opposite v on this edge."""
            return self._destination if v is self._origin else self._origin
    
        def element(self):
            """Return element associated with this edge."""
            return self._element
        
        def __hash__(self):     # will allow edge to be a map/set key
            return hash((self._origin, self._destination))
        
    # -------------------------------------- #
    
    def __init__(self, directed = False):
        """Create an empty graph (undirected, by default).
            Graph is directed if optional parameter is set to True.
        """
        self._outgoing = {}
        # only create second map for directed graph; use alias for undirected
        self._incoming = {} if directed else self._outgoing
        
    def is_directed(self):
        """Return True if this is a directed graph; False if undirected.
            Property is based on the original declaration of the graph, not its contents.
        """
        return self._incoming is not self._outgoing    # directed if maps are distinct
    
    def vertex_count(self):
        """Return the number of vertices in the graph."""
        return len(self._outgoing)
    
    def vertices(self):
        """Return an iteration of all vertices of the graph."""
        return self._outgoing.keys()
    
    def edge_count(self):
        """Return the number of edges in the graph."""
        total = sum(len(self._outgoing[v]) for v in self._outgoing)
        # for undircted graphs, make sure not to double-count edges
        return total if self.is_directed() else total // 2
    
    def edges(self):
        """Return a set of all edges of the graph."""
        result = set()    # avoid double-reproting edges of undirected graph
        for secondary_map in self._outgoing.values():
            result.update(secondary_map.values())     # add edges to resulting set
        return result
    
    def get_edge(self, u, v):
        """Return the edge from u to v, or None if not adjacent."""
        return self._outging[u].get(v)     # returns None if v not adjacent
    
    def degree(self, v, outgoing = True):
        """Return number of (outgoing) edges incident to vertex v in the graph.
            If graph is directed, optional parameter used to count incoming edges.
        """
        adj = self._outgoing if outgoing else self._incoming
        return len(adj[v])
    
    def incident_edges(self, v, outgoing = True):
        """Return all (outgoing) edges incident to vertex v in the graph.
            If graph is directed, optional parameter used to request incoming edges.
        """
        adj = self._outgoing if outgoing else self._incoming
        for edge in adj[v].values():
            yield edge
            
    def insert_vertex(self, x = None):
        """Insert and return a new Vertex with element x."""
        v = self.Vertex(x)
        self._outgoing[v] = {}
        if self.is_directed():
            self._incoming[v] = {}     # need distinct map for incoming edges
        return v
    
    def insert_edge(self, u, v, x = None):
        """Insert and return a new Edge from u to v with auxiliary element x."""
        e = self.Edge(u, v, x)
        self._outgoing[u][v] = e
        self._incoming[v][u] = e    
        
        




In [ ]:
### Depth Frist Search ###

def DFS(g, u, discovered):
    """Perform DFS of the undiscovered portion of Graph g starting at Vertex u.
        
        1) g is the graph storing all vertices and edges.
        2) discovered is a dictionary mapping each vertex to the edge  
            that was used to discover it during the DFS. 
            (u should be 'discovered' prior to the call.) 
        3) Newly discovered vertices will be added to the dictionary as a result.
    """
    
    for e in g.incident_edges(u):     # for every outgoing edge from u
        v = e.opposite(u)
        if v not in discovered:       # v is an unvisisted vertex
            discovered[v] = e         # e is the tree edge that discovered v
            DFS(g, v, discovered)     # recursively explore from v

            
# --- Initialize with a third party parameter: discovered --- #
# 1) discovered is used to maps a vertex of the graph to the tree edge 
# that was used to discover that vertex.
# 2) Assume that the source vertex u occurs as a key of the dictionary, 
#     , with None as its value.
# 3) A caller need to start the traversal as follows.
# ----------------------------------------------------------- #

result = {u: None}     # a new dictionary, with u trivially discovered
DFS(g, u, result)


### Reconstructing a Path from u to v ###

def construct_path(u, v, discovered):
    path = []                          # empty path by default
    if v in discovered:
        # build list from v to u and then reverse it at the end
        path.append(v)
        walk = v
        while walk is not u:
            e = discovered[walk]       # find edge leading to walk
            parent = e.opposite(walk)
            path.append(parent)
            walk = parent
        
        path.reverse()                 # reorient path from u to v
        return path
    
    
    
### Breath First Search ###

def BFS(g, s, discovered):
    """ Perfrom BFS of the undiscovered portion of Graph g starting at Vertex s.
        
        1) discovered is a dictionary mapping each vertex to the edge
            that was used to discovered it during the BFS 
            (s should be mapped to None prior to the call).
        2) Newly discovered vertices will be added to the dictionary as a result.
    """
    level_count = 0
    level = [s]                             # first level includes only s
    while len(level) > 0:
        next_level = []                     # prepare to gather newly found vertices
        for u in level:
            for e in g.incident_edges(u):   # for every outgoing edge from u
                v = e.opposite(u)
                if v not in discovered:     # v is an unvisted vertex
                    discovered[v] = e       # e is the tree edge that discovered v
                    next_level.append(v)    # v will be further considered in next pass
        level = next_level                  # relabel 'next' level to become current
        level_count += 1
                
        

In [31]:
### Topological Sort ###

def topological_sort(graph):
    """ Return a list of verticies of directed acyclic graph in topological order.
        If graph has a cycle, the result will in be incomplete.
        The subgraph of the vertices that have not been ordered contain a directed cycle.
    """
    
    topo_order = []  # a list of vertices in topological order
    to_visit = []   # a list of vertices that have no incoming vertices unvisited (free of contraints)
    in_count = {}   # keep track of in-degree for each vertex
    
    for u in graph.vertices():
        in_count[u] = graph.degree(u, False)  # parameter requests incoming degree
        if in_count[u] == 0:                  # if u has no incoming edges,
            to_visit.append(u)                # it is free of constraints (add it to the to_visit list)
    
    while len(to_visit) > 0:
        current = to_visit.pop()              # current vertex is free of constraints
        topo_order.append(current)            # add current to the topological order list
        
        for neighbor in graph.incident_edges(current):  # consider all outgoing neighbors of current vertex
            v = neighbor.opposite(current)
            in_count[v] -= 1                  # v has one less constraint without current vertex (when current vertex is visited)
            if in_count[v] == 0:
                to_visit.append(v)            # if v is free of constraints, add it to the to_visit list

    return topo_order


def test_acyclic(graph):
    """ Return Ture if the graph is acyclic; otherwise, return Flase.
    """
    topo = topological_sort(graph)
    #print('topo lens: %d ; vertex lens: %d' % (len(topo),graph.vertex_count()))
    return len(topo) == graph.vertex_count()       

        

In [37]:
#####################################################
# HW 3: Detect Cycle in a Driected Graph
# print yes if there is a cycle; otherwise print no.
#####################################################

test_case = int(input())

for case in range(test_case):
    (vertex, edge) = [int(e) for e in input().split()]
    g = Graph(directed = True)
    
    for i in range(edge):
        (a, b) = [int(e) for e in input().split()]
        (A, B) = (None, None)
        
        for vertex in g.vertices():
            if vertex.element() == a:
                A = vertex
            elif vertex.element() == b:
                B = vertex
        if A == None:
            A = g.insert_vertex(a) 
        if B == None:
            B = g.insert_vertex(b) 
        g.insert_edge(A, B)
    
    # print(topological_sort(g))
    print('No') if test_acyclic(g) else print('Yes')
    

2
3 2
1 2
2 3
No
3 3
1 2
2 3
3 1
Yes
